In [1]:
import numpy as np
import tensorflow
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, InputLayer, BatchNormalization, Dropout, GlobalAveragePooling2D

In [2]:
hind = np.load('hindustani_MFCC_imagedata_766samples_19ragas.npy', allow_pickle=True, encoding='bytes')
carn = np.load('carnatic_MFCC_imagedata_1175.npy', allow_pickle=True, encoding='bytes')
dataset = np.concatenate((hind, carn), axis=0)
n = len(dataset)

x = dataset[:, :-1]
y = dataset[:, -1]

g = []
for i in range(n):
    a=x[i][0]
    g.append(a)
g = np.array(g)

import category_encoders as ce
enc = ce.OneHotEncoder(y, handle_unknown='return_nan', return_df=True, use_cat_names=True)
y_en = enc.fit_transform(y)

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(g, y_en, test_size=0.2, random_state=0)

MemoryError: Unable to allocate 3.15 GiB for an array with shape (1941, 576, 1008, 3) and data type uint8

In [8]:
cnn = Sequential()

cnn.add(Conv2D(50, kernel_size=(3,3), strides=(9,9), padding='same', activation='relu', input_shape=(576, 1008, 3)))

cnn.add(Conv2D(75, kernel_size=(3,3), strides=(9,9), padding='same', activation='relu'))
cnn.add(MaxPool2D(pool_size=(2,2)))
cnn.add(Dropout(0.25))

cnn.add(Conv2D(105, kernel_size=(3,3), strides=(9,9), padding='same', activation='relu'))
# cnn.add(MaxPool2D(pool_size=(2,2)))
cnn.add(Dropout(0.25))

cnn.add(Conv2D(150, kernel_size=(3,3), strides=(9,9), padding='same', activation='relu'))
# cnn.add(MaxPool2D(pool_size=(2,2)))
cnn.add(Dropout(0.25))

cnn.add(Conv2D(170, kernel_size=(3,3), strides=(9,9), padding='same', activation='relu'))
# cnn.add(MaxPool2D(pool_size=(2,2)))
cnn.add(Dropout(0.25))

cnn.add(GlobalAveragePooling2D())
# cnn.add(Flatten())

cnn.add(Dense(150, activation='relu'))
cnn.add(Dropout(0.4))
cnn.add(Dense(80, activation='relu'))
cnn.add(Dropout(0.3))


cnn.add(Dense(55, activation='softmax'))

In [9]:
cnn.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_19 (Conv2D)           (None, 64, 112, 50)       1400      
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 8, 13, 75)         33825     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 4, 6, 75)          0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 4, 6, 75)          0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 1, 1, 105)         70980     
_________________________________________________________________
dropout_10 (Dropout)         (None, 1, 1, 105)         0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 1, 1, 150)        

In [10]:
cnn.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [11]:
cnn.fit(x_train, y_train, batch_size=8, epochs=50, validation_data=(x_test, y_test), verbose=1)

Epoch 1/50
194/194 [==============================] - 612s 3s/step - loss: 4.0498 - accuracy: 0.0213 - val_loss: 4.0070 - val_accuracy: 0.0154
Epoch 2/50
194/194 [==============================] - 509s 3s/step - loss: 4.0066 - accuracy: 0.0219 - val_loss: 4.0065 - val_accuracy: 0.0308
Epoch 3/50
194/194 [==============================] - 371s 2s/step - loss: 4.0048 - accuracy: 0.0245 - val_loss: 4.0065 - val_accuracy: 0.0154
Epoch 4/50
194/194 [==============================] - 468s 2s/step - loss: 4.0044 - accuracy: 0.0271 - val_loss: 4.0060 - val_accuracy: 0.0334
Epoch 5/50
194/194 [==============================] - 415s 2s/step - loss: 4.0031 - accuracy: 0.0245 - val_loss: 4.0058 - val_accuracy: 0.0334
Epoch 6/50
194/194 [==============================] - 389s 2s/step - loss: 4.0022 - accuracy: 0.0238 - val_loss: 4.0055 - val_accuracy: 0.0334
Epoch 7/50
194/194 [==============================] - 438s 2s/step - loss: 4.0014 - accuracy: 0.0238 - val_loss: 4.0058 - val_accuracy: 0.0334

In [12]:
dataset

array([[array([[[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       ...,

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 25

In [15]:
g[600].shape

(576, 1008, 3)

In [16]:
def img_3d_to_2d(np_arr_3d):
  gray = [0.299, 0.587, 0.114]
  arr_2d = np.dot(np_arr_3d, gray)
  return arr_2d  

In [17]:
h=[]

In [ ]:
for i in range(n):
    arr_3d = g[i]
    arr_2d = img_3d_to_2d(arr_3d)
    h.append(arr_2d)